In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/train.csv')
print(df['Model'].nunique())
print(df['State'].nunique())
print(df['City'].nunique())
print(df['Make'].nunique())
print(df['Year'].nunique())

2094
59
2524
56
22


In [ ]:
class Linear_Regression:
  def __init__(self, x, target, columns, learning_rate, epoch, non_numeric_columns, convert_to_vector):
    self.columns = columns
    self.x = x
    self.target = target
    self.learning_rate = learning_rate
    self.epoch = epoch
    self.non_numeric_columns = non_numeric_columns
    self.non_numeric_dic = {}
    self.convert_to_vector = convert_to_vector
    self.convert_to_vector_value = {}
    self.final_columns = self.columns.copy()

  def one_hot_encode(self, dictionary, values):
    for x in values:
        if x not in dictionary.keys():
            dictionary[x] = len(dictionary)

  def one_hot_decode(self, dictionary, key):
    if key in dictionary.keys():
        return dictionary[key]
    dictionary[key] = len(dictionary)
    return dictionary[key]

  def convert_to_numeric(self):
    for i in self.non_numeric_columns:
      dictionary = {}
      self.non_numeric_dic[i] = dictionary
      self.one_hot_encode(dictionary, self.x[i].values)
      self.x[i] = [self.one_hot_decode(dictionary, j) for j in self.x[i].values]

  def normalize(self):
    self.normalize_dictionary = {}
    for i in self.final_columns:
      if i in self.convert_to_vector:
        continue
      mean = np.mean(self.x[i])
      var = np.var(self.x[i])
      self.x[i] = (self.x[i] - mean) / var
      self.normalize_dictionary[i] = {'mean':mean, 'var':var}
  
  def gradian_function(self, predict, w, x_vector):
    return np.dot(predict - self.target, x_vector) / len(x_vector)

  def cost_function_simplified(self, predicts):
    return np.sum((predicts - self.target) ** 2) / (2 * len(self.target))

  def convert_columns_to_Vector(self):
    for i in self.convert_to_vector:
      if i in self.final_columns:
        self.convert_to_vector_value[i] = self.x[i].nunique()
        self.final_columns.remove(i)
      count = self.convert_to_vector_value[i]
      for j in range(count):
        self.final_columns.append(i + str(j))
        self.x[i + str(j)] = np.where(self.x[i] == j, 1, 0)

  def LR_GD(self):
    self.convert_to_numeric()
    self.normalize()
    self.convert_columns_to_Vector()
    x_vector = self.x.loc[:, self.final_columns].to_numpy()
    # Adding A New Column With Value 1, Just To Set The Bias Along Side With W
    bias = np.ones(len(self.x))
    x_vector = np.c_[x_vector, bias]
    w = [random.random() for x in range(len(self.final_columns) + 1)]
    # w = [random.random() for x in range(len(columns))]
    # w.append(np.mean(self.target))
    predict = np.dot(x_vector, w)
    costs = []
    cost = self.cost_function_simplified(predict)
    costs.append(cost)
    for i in range(self.epoch):
      gradian = self.gradian_function(predict, w, x_vector)
      w = w - (self.learning_rate * gradian)
      predict = np.dot(x_vector, w)
      predict = np.array(predict)
      cost = self.cost_function_simplified(predict)
      costs.append(cost)
    plt.plot([i for i in range(len(costs))], costs)
    plt.xlabel('Test Number')
    plt.ylabel('Cost Function')
    plt.title('Cost-Function-Result')
    plt.savefig("Cost-Function-Image.png")
    plt.show()
    return w

  def test(self, x_vector, target, w, show_plot = True, path = ''):
    for col in self.non_numeric_columns:
      dic = self.non_numeric_dic[col]
      x_vector[col] = [self.one_hot_decode(dic, i) for i in x_vector[col].values]

    for col in self.columns:
      if col in self.convert_to_vector:
        continue
      dic = self.normalize_dictionary[col]
      x_vector[col] = [(i - dic['mean']) / dic['var'] for i in x_vector[col]]

    for i in self.convert_to_vector:
      count = self.convert_to_vector_value[i]
      for j in range(count):
        x_vector[i + str(j)] = np.where(x_vector[i] == j, 1, 0)

    x_matrix = x_vector.loc[:, self.final_columns].to_numpy()
    bias_matrix = np.ones(len(x_vector))
    x_matrix = np.c_[x_matrix, bias_matrix]
    predict = np.absolute(np.dot(x_matrix, w))
    if not show_plot:
      df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/test.csv')
      df['Price'] = predict
      df.loc[:,['Price', 'CarId']].to_csv('/content/predict.csv', index = False)
      return
    plt.plot([x for x in range(len(target))], np.array(target), color='b')
    plt.plot([x for x in range(len(predict))], np.array(predict), color='r')
    plt.savefig('Test.png')
    plt.show()
    mins = [min(target), min(predict)]
    maxs = [max(target), max(predict)]
    plt.plot([min(mins), max(maxs)], [min(mins), max(maxs)])
    plt.scatter(predict, target)
    plt.xlabel('Predicted Price')
    plt.ylabel('Price')
    plt.savefig('Professional-Result.png')
    plt.show()

In [ ]:
############################################# My Test #############################################
df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/train.csv')
test = df.iloc[99900:, :]
df = df.iloc[:99900, :]
# columns = ['Year', 'Mileage', 'City', 'State', 'Make', 'Model']
# non_numeric_columns = ['City', 'State', 'Make', 'Model']
columns = ['Year', 'Mileage', 'State', 'Make', 'Model']
non_numeric_columns = ['State', 'Make', 'Model']
vectorize = ['Make', 'State', 'Year']
target = df['Price']
lr = Linear_Regression(df.loc[:,columns], target, columns, 0.0175, 50000, non_numeric_columns, vectorize)
w = lr.LR_GD()
print('W: ', w)
lr.test(test.loc[:,columns], test.loc[:,['Price']].values, w)

In [1]:
############################################# Upload #############################################
df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/train.csv')
target = df['Price']
test_path = '/content/drive/MyDrive/ML_HW2_Q1/test.csv'
test = pd.read_csv(test_path)
# columns = ['Year', 'Mileage', 'City', 'State', 'Make', 'Model']
# non_numeric_columns = ['City', 'State', 'Make', 'Model']
columns = ['Year', 'Mileage', 'State', 'Make', 'Model']
non_numeric_columns = ['State', 'Make', 'Model']
#####   Best Answer  #####
convert_to_vector = ['State', 'Make']
# convert_to_vector = ['State', 'Make', 'Year']
# convert_to_vector = ['City', 'State', 'Make', 'Model']
show_plot = False
#####   Best Answer  #####
lr = Linear_Regression(df.loc[:,columns], target, columns, 0.018888885, 100000, non_numeric_columns, convert_to_vector)
w = lr.LR_GD()
print('W: ', w)
lr.test(test.loc[:,columns], [], w, show_plot, test_path)

NameError: ignored

In [ ]:
# ////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////////

# def one_hot_encode(dictionary, df):
#     for x in df:
#         if x not in dictionary.keys():
#             dictionary[x] = len(dictionary)

# def one_hot_decode(dictionary, key):
#     if key in dictionary.keys():
#         return dictionary[key]
#     dictionary[key] = len(dictionary)
#     return dictionary[key]


# # def loss_function(predicted, target):
# #     return ((predicted - target) ** 2) / 2
# #
# # def cost_function(predicts, targets):
# #     return sum([loss_function(predicts[x], targets[x]) for x in range(len(targets))]) / len(targets)

# def cost_function_simplified(predicts, targets):
#     return np.sum((predicts - targets) ** 2) / (2 * len(targets))

# def gradian_function(predict, target, w, x_vector):
#     return np.dot(predict - target, x_vector) / len(x_vector)
#     # diff = predict - target
#     # gradian = np.dot(diff, x_vector)
#     # return gradian / len(x_vector)

# # df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/train.csv')
# df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/train.csv').iloc[:99900, :]

# # Converting Non Numeric Values To Numeric Values By Using one_hot_function
# city_rank = {}
# one_hot_encode(city_rank, df['City'])
# rank = [city_rank[x] for x in df['City']]
# city_mean = np.mean(rank)
# city_var = np.var(rank)
# df['City Rank'] = [(x - city_mean) / city_var for x in rank]

# state_rank = {}
# one_hot_encode(state_rank, df['State'])
# rank = [state_rank[x] for x in df['State']]
# state_mean = np.mean(rank)
# state_var = np.var(rank)
# df['State Rank'] = [(x - state_mean) / state_var for x in rank]

# # vin_rank = {}
# # one_hot_encode(vin_rank, df['Vin'])
# # rank = [vin_rank[x] for x in df['Vin']]
# # vin_mean = np.mean(rank)
# # vin_var = np.var(rank)
# # df['Vin Rank'] = [(x - vin_mean) / vin_var for x in rank]

# make_rank = {}
# one_hot_encode(make_rank, df['Make'])
# rank = [make_rank[x] for x in df['Make']]
# make_mean = np.mean(rank)
# make_var = np.var(rank)
# df['Make Rank'] = [(x - make_mean) / make_var for x in rank]

# model_rank = {}
# one_hot_encode(model_rank, df['Model'])
# rank = [model_rank[x] for x in df['Model']]
# model_mean = np.mean(rank)
# model_var = np.var(rank)
# df['Model Rank'] = [(x - model_mean) / model_var for x in rank]

# year_mean = np.mean(df['Year'])
# year_var = np.var(df['Year'])
# df['Year'] = [(x - year_mean) / year_var for x in df['Year']]

# mileage_mean = np.mean(df['Mileage'])
# mileage_var = np.var(df['Mileage'])
# df['Mileage'] = [(x - mileage_mean) / mileage_var for x in df['Mileage']]


# # columns = ['Year', 'Mileage', 'City Rank', 'State Rank', 'Vin Rank', 'Make Rank', 'Model Rank']
# columns = ['Year', 'Mileage', 'City Rank', 'State Rank', 'Make Rank', 'Model Rank']
# x_vector = df.loc[:, columns].to_numpy()

# # Adding A New Column With Value 1, Just To Set The Bias Along Side With W
# bias = np.ones(len(df))
# x_vector = np.c_[x_vector, bias]
# # print('x_vector:')
# # print(x_vector[0])

# target = np.array(df.iloc[:,0].values)
# # target_mean = np.mean(target)
# # target_var = np.var(target)
# # target = (target - target_mean) / target_var
# # print('target:')
# # print(target)

# learning_rate = 0.0015
# epoch = 30000

# # Setting The Bias Number In This Vector Along Side Of Xi
# # w = [((random.random() * 1)) for x in range(len(columns) + 1)]
# w = [((random.random() * 1) + 0) for x in range(len(columns) + 1)]
# # w = [((random.random() * 1) + 0) for x in range(len(columns))]
# # w.append(np.mean(df['Price']))
# # w.append(np.mean(df['Price']))
# # w.append(1)
# # w.append(df['Price'].mean())
# # w = [1 for x in range(len(columns) + 1)]
# # print('W:')
# # print(w)


# predict = np.dot(x_vector, w)
# # print('predict:')
# # print(predict)
# cost = cost_function_simplified(predict, target)
# # print('cost:')
# # print(cost)

# coasts = []
# for i in range(epoch):
#     gradian = gradian_function(predict, target, w, x_vector)
#     w = w - (learning_rate * gradian)
#     # print('New W:')
#     # print(w)
#     predict = np.dot(x_vector, w)
#     # print('Predict:')
#     # print(predict)
#     cost = cost_function_simplified(predict, target)
#     # print('cost:')
#     # print(cost)
#     coasts.append(cost)

# x = [i for i in range(len(coasts))]
# # plt.plot(x, coasts, marker = 'o', mec = 'r', mfc = 'r', linestyle  = "--")
# plt.plot(x, coasts)
# plt.xlabel('Test Number')
# plt.ylabel('Cost Function')
# plt.title('Cost Function Result')
# plt.savefig("Loss-Function-Plot.png")
# plt.show()


# ###################################################################
# ##############         My Test          ####################
# df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/train.csv').iloc[99900:, :]

# # Converting Non Numeric Values To Numeric Values By Using rank function
# df['City Rank'] = [(one_hot_decode(city_rank, x) - city_mean) / city_var for x in df['City']]
# # df['City Rank'] = [one_hot_decode(city_rank, x) for x in df['City']]
# # df['City Rank'] = [(x - np.mean(df['City Rank'])) / np.var(df['City Rank']) for x in df['City Rank']]
# df['State Rank'] = [(one_hot_decode(state_rank, x) - state_mean) / state_var for x in df['State']]
# # df['State Rank'] = [one_hot_decode(state_rank, x) for x in df['State']]
# # df['State Rank'] = [(x - np.mean(df['State Rank'])) / np.var(df['State Rank']) for x in df['State Rank']]
# # df['Vin Rank'] = [(one_hot_decode(vin_rank, x) - vin_mean) / vin_var for x in df['Vin']]
# df['Make Rank'] = [(one_hot_decode(make_rank, x) - make_mean) / make_var for x in df['Make']]
# # df['Make Rank'] = [one_hot_decode(make_rank, x) for x in df['Make']]
# # df['Make Rank'] = [(x - np.mean(df['Make Rank'])) / np.var(df['Make Rank']) for x in df['Make Rank']]
# df['Model Rank'] = [(one_hot_decode(model_rank, x) - model_mean) / model_var for x in df['Model']]
# # df['Model Rank'] = [one_hot_decode(model_rank, x) for x in df['Model']]
# # df['Model Rank'] = [(x - np.mean(df['Model Rank'])) / np.var(df['Model Rank']) for x in df['Model Rank']]

# df['Year'] = [(x - year_mean) / year_var for x in df['Year']]

# df['Mileage'] = [(x - mileage_mean) / mileage_var for x in df['Mileage']]

# test_x = df.loc[:, columns].to_numpy()
# test_bias = np.ones(len(df))
# test_x = np.c_[test_x, test_bias]

# df['Predicted Price'] = np.dot(test_x, w)
# # df['Predicted Price'] = [abs(x) for x in df['Predicted Price']]
# # df['Predicted Price'] = [(x * target_var) + target_mean for x in df['Predicted Price']]

# print('Total Predicted Price:')
# print(df['Predicted Price'].to_string())
# print('w:')
# print(w)
# print('x_vector:')
# print(test_x)

# plt.subplot(211)
# plt.plot(df['Price'])
# plt.subplot(212)
# plt.plot(df['Predicted Price'])
# plt.show()

# plt.plot([x for x in range(len(df))], np.array(df.iloc[:,0].values), color='b')
# plt.plot([x for x in range(len(df))], np.array(df.loc[:,['Predicted Price']].values), color='r')
# plt.savefig('My Test On 10 Test date.png')
# plt.show()
# # print('Target Mean:')
# # print(target_mean)
# # print('Target Var:')
# # print(target_var)
# mins = [min(df['Predicted Price']), min(df['Price'])]
# maxs = [max(df['Predicted Price']), max(df['Price'])]
# plt.plot([min(mins), max(maxs)], [min(mins), max(maxs)])
# plt.scatter(df['Predicted Price'], df['Price'])
# plt.xlabel('Predicted Price')
# plt.ylabel('Price')
# plt.savefig('Professional-Result.png')
# plt.show()
# print('Cost:')
# print(cost_function_simplified(df['Predicted Price'], df['Price']))
# exit(0)
# ###################################################################
# # df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/test.csv')
# # # just in case that we can see the result, if we get all test.csv file, we get error of merror space due to its huge size
# # # df = pd.read_csv('/content/train.csv')

# # # Converting Non Numeric Values To Numeric Values By Using rank function
# # df['City Rank'] = [(one_hot_decode(city_rank, x) - city_mean) / city_var for x in df['City']]
# # df['State Rank'] = [(one_hot_decode(state_rank, x) - state_mean) / state_var for x in df['State']]
# # # df['Vin Rank'] = [(one_hot_decode(vin_rank, x) - vin_mean) / vin_var for x in df['Vin']]
# # df['Make Rank'] = [(one_hot_decode(make_rank, x) - make_mean) / make_var for x in df['Make']]
# # df['Model Rank'] = [(one_hot_decode(model_rank, x) - model_mean) / model_var for x in df['Model']]


# # # df['City Rank'] = [one_hot_decode(city_rank, x) for x in df['City']]
# # # df['State Rank'] = [one_hot_decode(state_rank, x) for x in df['State']]
# # # # df['Vin Rank'] = [(one_hot_decode(vin_rank, x) - vin_mean) / vin_var for x in df['Vin']]
# # # df['Make Rank'] = [one_hot_decode(make_rank, x) for x in df['Make']]
# # # df['Model Rank'] = [one_hot_decode(model_rank, x) for x in df['Model']]

# # # mean = np.mean(df['City Rank'])
# # # var = np.var(df['City Rank'])
# # # df['City Rank'] = [(x - mean) / var for x in df['City Rank']]
# # # mean = np.mean(df['State Rank'])
# # # var = np.var(df['State Rank'])
# # # df['State Rank'] = [(x - mean) / var for x in df['State Rank']]
# # # mean = np.mean(df['Make Rank'])
# # # var = np.var(df['Make Rank'])
# # # df['Make Rank'] = [(x - mean) / var for x in df['Make Rank']]
# # # mean = np.mean(df['Model Rank'])
# # # var = np.var(df['Model Rank'])
# # # df['Model Rank'] = [(x - mean) / var for x in df['Model Rank']]
# # # df['Vin Rank'] = [(one_hot_decode(vin_rank, x) - vin_mean) / vin_var for x in df['Vin']]


# # df['Year'] = [(x - year_mean) / year_var for x in df['Year']]

# # df['Mileage'] = [(x - mileage_mean) / mileage_var for x in df['Mileage']]

# # # print(df.head().to_string())

# # # columns = ['Year', 'Mileage', 'City Rank', 'State Rank', 'Vin Rank', 'Make Rank', 'Model Rank']
# # test_x = df.loc[:, columns].to_numpy()
# # test_bias = np.ones(len(df))
# # test_x = np.c_[test_x, test_bias]

# # df['Price'] = np.dot(test_x, w)
# # # print(df.to_string())
# # # res_df = pd.read_csv('/content/drive/MyDrive/ML_HW2_Q1/test.csv')
# # # res_df['Price'] = df['Price']
# # # print(res_df.to_string())
# # df.loc[:,['Price', 'CarId']].to_csv('/content/predict.csv', index = False)



# ########################### SK_Learn #####################
# # df = pd.read_csv('/content/train.csv')
# # non_numeric = ['Model', 'Make', 'Vin', 'State', 'City']
# # df['Model'] = [one_hot_decode(model_rank, x) for x in df['Model']]
# # df['Make'] = [one_hot_decode(make_rank, x) for x in df['Make']]
# # df['Vin'] = [one_hot_decode(vin_rank, x) for x in df['Vin']]
# # df['State'] = [one_hot_decode(state_rank, x) for x in df['State']]
# # df['City'] = [one_hot_decode(city_rank, x) for x in df['City']]
# # from sklearn.linear_model import LinearRegression
# # lr_model = LinearRegression()
# # lr_model.fit(df.iloc[:,1:], df.iloc[:,0])
# # w = lr_model.coef_
# # b = lr_model.intercept_
# # df['Predicted-Price'] = np.dot(df.iloc[:,1:], w) + b
# # mins = [min(df['Price']), min(df['Predicted-Price'])]
# # maxs = [max(df['Price']), max(df['Predicted-Price'])]
# # plt.plot([min(mins), max(maxs)], [min(mins), max(maxs)])
# # plt.scatter(df['Price'], df['Predicted-Price'])
# # plt.title('AAAAAAAAAAAAAAAAAAAAAAAA')
# # plt.xlabel('Price')
# # plt.ylabel('sk_learn Price')
# # plt.savefig('sk_learn-MyPredict.png')
# # plt.show()
# # test_df = pd.read_csv('/content/test.csv')
# # test_df['Model'] = [one_hot_decode(model_rank, x) for x in test_df['Model']]
# # test_df['Make'] = [one_hot_decode(make_rank, x) for x in test_df['Make']]
# # test_df['Vin'] = [one_hot_decode(vin_rank, x) for x in test_df['Vin']]
# # test_df['State'] = [one_hot_decode(state_rank, x) for x in test_df['State']]
# # test_df['City'] = [one_hot_decode(city_rank, x) for x in test_df['City']]

# # test_df['Price'] = np.dot(test_df.iloc[:,1:], w) + b
# # mins = [min(res_df['Price']), min(test_df['Price'])]
# # maxs = [max(res_df['Price']), max(test_df['Price'])]
# # plt.plot([min(mins), max(maxs)], [min(mins), max(maxs)])
# # plt.scatter(res_df['Price'], test_df['Price'])
# # plt.xlabel('My Price')
# # plt.ylabel('sk_learn Price')
# # plt.savefig('sk_learn-MyPredict.png')
# # plt.show()

TypeError: ignored